In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0,'..')

In [3]:
import numpy as np
from old import tools, RNTK_avg
import jax
import symjax
import symjax.tensor as T
import copy

In [4]:
Ti = 2
TiPrime = 2

In [5]:
dataset = "trains"

In [6]:
dic = {k:v for k,v in map(lambda x : x.split(), open("../data" + "/" + dataset + "/" + dataset + ".txt", "r").readlines())}

In [7]:
N = int(dic["n_patrons1="])
length = int(dic["n_entradas="])

In [11]:
class RNTKTEST():
    def __init__(self):
        self.sw = 1
        self.su = 1
        self.sb = 1
        self.sh = 1
        self.L = 1
        self.Lf = 0
        self.sv = 1
        self.N = int(dic["n_patrons1="])
        self.length = int(dic["n_entradas="])
        
    def VT(self, M):
        A = T.diag(M)  # GP_old is in R^{n*n} having the output gp kernel
        # of all pairs of data in the data set
        B = A * A[:, None]
        C = T.sqrt(B)  # in R^{n*n}
        D = M / C  # this is lamblda in ReLU analyrucal formula
        E = T.clip(D, -1, 1)  # clipping E between -1 and 1 for numerical stability.
        F = (1 / (2 * np.pi)) * (E * (np.pi - T.arccos(E)) + T.sqrt(1 - E ** 2)) * C
        G = (np.pi - T.arccos(E)) / (2 * np.pi)
        return F,G

In [8]:
param = {'sigmaw':1, 'sigmau':1, 'sigmab':1, 'sigmah':1, 'L':1, 'Lf':1, 'sigmav':1}

In [19]:
rntk = RNTK_avg.RNTK(N,length,param).RNTK_function()

N, 10, length, 29
gp_new 1 Op(name=expand_dims_35, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)
rntk_new 1 Op(name=expand_dims_35, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)
gp_new 2 Op(name=concatenate_20, fn=concatenate, shape=(2, 10, 10), dtype=float32, scope=/default)
rntk_new 2 Op(name=concatenate_21, fn=concatenate, shape=(2, 10, 10), dtype=float32, scope=/default)
gp_new 3 Op(name=expand_dims_38, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)
rntk_new 3 Op(name=expand_dims_39, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)
gp_new 4 Op(name=concatenate_22, fn=concatenate, shape=(2, 10, 10), dtype=float32, scope=/default)
rntk_new 4 Op(name=concatenate_23, fn=concatenate, shape=(2, 10, 10), dtype=float32, scope=/default)


In [15]:
rntk[0]

Op(name=add_24, fn=add, shape=(10, 10), dtype=float32, scope=/default)

In [13]:
DATA = T.Placeholder((N, length), 'float32', name = "X")
x = DATA[:,0]
X = x*x[:, None]
n = X.shape[0]

In [14]:
RNTK,GP = RNTK_avg.RNTK.RNTK_first(rntk,DATA[:,0])

In [15]:
RNTK

Op(name=concatenate_1, fn=concatenate, shape=(2, 10, 10), dtype=float32, scope=/default)

## LAYER ZERO ATTEMPT


In [11]:
switch_flag = 1
dim_1_i = dim_1 = 2
dim_2 = 2
dim_2_i = 0
dim_num = dim_1 + dim_2 + 1

In [12]:
tiprimes = []
tis = []

print(dim_1_i, dim_2_i, switch_flag)
for d in range(0,dim_num):
    tiprime = dim_1_i
    ti = dim_2_i

    # diag_func(tiprime, ti, dim_1, dim_2, rntk)
    tiprimes.append(tiprime)
    tis.append(ti)

    if dim_1_i == 0:
        switch_flag -= 1
    else:
        dim_1_i = dim_1_i - 1
    if switch_flag <= 0:
        dim_2_i = dim_2_i + 1

2 0 1


In [13]:
np.array(list(zip(tiprimes, tis))).shape ## input size (iterable)

(5, 2)

In [14]:
np.array([dim_1, dim_2, n, X]).shape ## additional input size (unchanging)

(4,)

In [15]:
-np.ones([dim_num,min(dim_1,dim_2)], dtype = object) ## the carry

array([[-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1]], dtype=object)

In [16]:
w = T.Variable(1.0, dtype="float32")
v = T.Placeholder((), "float32")
out = T.while_loop(
    lambda i, u: i[0] + u < 5,
    lambda i: (i[0] + 1.0, i[0] ** 2),
    (w, 1.0),
    non_sequences_cond=[v],
)

In [17]:
        # print("inner iteration - ", tiprime_iter, ti_iter) #// DIM 1 IS T PRIME, DIM 2 IS T
        # if T.logical_and(T.greater(ti_iter, 0), T.greater(tiprime_iter, 0)):
        # # if ((ti_iter > 0) & (tiprime_iter > 0)):
        #     # S, D = rntk.VT(lambdamatrix[0, ti_iter-1, tiprime_iter-1])
        #     # lambdamatrix[0, ti_iter, tiprime_iter] = rntk.sw ** 2 * S + rntk.su ** 2 * X + rntk.sb ** 2
        #     # phimatrix[0, ti_iter, tiprime_iter] = lambdamatrix[0, ti_iter, tiprime_iter] + rntk.sw ** 2 * phimatrix[0, ti_iter - 1, tiprime_iter - 1] * D
        #     T.index_update(carry, it, 2)
        # else:
        #     # test = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + (rntk.su ** 2) * X + rntk.sb ** 2
        #     # test = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + (rntk.su ** 2) + rntk.sb ** 2
        #     # test = rntk.sh ** 2 * rntk.sw ** 2 + (rntk.su ** 2) + rntk.sb ** 2
        #     # phimatrix[0,ti,tiprime_iter] = lambdamatrix[0,ti,tiprime_iter] = T.expand_dims(test, axis = 0) # line 2, alg 1
        #     # phi_array[0,ti_iter,tiprime_iter] = 1 # line 2, alg 1
        #     T.index_update(carry, it, 3)

In [18]:
def body_func(inp):
    print("in body function")
    carry = inp[4]
    tiprime_iter = inp[0] + 1
    ti_iter = inp[1] + 1
    return (tiprime_iter, ti_iter, inp[2], inp[3], carry)

In [19]:
def cond_funct(inp):
    print("in cond function")
    tip_iter = inp[0]
    ti_iter = inp[1]
    dim1max = inp[2]
    dim2max = inp[3]
    return T.logical_and(T.less_equal(tip_iter, dim1max), T.less_equal(ti_iter, dim2max))

In [20]:
# Xph = T.Placeholder((10,10), "float32", name="X")
signal = T.Placeholder((5,2), "float32", name="signal")
dim1ph = T.Variable(dim_1, "float32", "alpha")
dim2ph = T.Variable(dim_2, "float32", "alpha")
nph = T.Variable(n, "float32", "alpha")
# carry = T.Placeholder((), "float32", "alpha")

def fn(ct, st, d1ph, d2ph, nph):
# def fn(ct, st, d1ph, d2ph, nph, x):
    print(ct)
    print(st)
    ct_new = copy.copy(ct)
    output = T.while_loop(
        lambda a: cond_funct(a), # condition
        lambda a: body_func(a), # body
        (st[0], st[1], d1ph, d2ph, ct), # init 
    )
    # print(ct)
    
    print("end of func def")
    return ct_new, output[4]
print("after func def")
last_ema, all_ema = T.scan(
    fn, init = -np.ones(np.min([dim_1,dim_2])), sequences=[signal], non_sequences=[dim1ph, dim2ph, nph]
)
print("after scan")
f = symjax.function(signal, dim1ph, dim2ph, nph, outputs=all_ema)

after func def
Placeholder(name=_5, shape=(2,), dtype=float32, scope=/default)
Placeholder(name=_6, shape=(2,), dtype=float32, scope=/default)
in cond function
in body function
end of func def


In [122]:
x = np.array(list(zip(tiprimes, tis)))
f(x, dim_1, dim_2, n)

Placeholder(name=_184, shape=(2,), dtype=float32, scope=/default)
Placeholder(name=_185, shape=(2,), dtype=int32, scope=/default)
[[2 0]
 [1 0]
 [0 0]
 [0 1]
 [0 2]]


array([[-1., -1.],
       [-1., -1.],
       [-1., -1.],
       [-1., -1.],
       [-1., -1.]], dtype=float32)

In [14]:
# lambdamatrix = np.zeros([1, dim_1 + 1, dim_2 + 1],dtype = object)
# phimatrix = np.zeros([1, dim_1 + 1, dim_2 + 1],dtype = object)

## MISC LOOKING AT OBJECT FORMATS

In [114]:
S_new,D_new = rntk.VT(gp_new[l-1])

In [115]:
S_new

Op(name=multiply_137, fn=multiply, shape=(10, 10), dtype=float32, scope=/default)

In [116]:
D_new

Op(name=true_divide_19, fn=true_divide, shape=(10, 10), dtype=float32, scope=/default)

In [117]:
extra = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + rntk.su**2 * S_new + rntk.sb**2

In [118]:
new = T.expand_dims(extra,axis = 0)

In [119]:
new

Op(name=expand_dims_27, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)

In [120]:
gp_new = T.concatenate([gp_new,new]) #line 4, alg 1

In [121]:
gp_new

Op(name=concatenate_13, fn=concatenate, shape=(3, 10, 10), dtype=float32, scope=/default)

## MIDDLE RNTK OBJECT TYPES

In [122]:
T.transpose(DATA[:, 1:])

Op(name=transpose_10, fn=transpose, shape=(28, 10), dtype=float32, scope=/default)

In [123]:
T.stack([RNTK,GP])

Op(name=stack_10, fn=stack, shape=(2, 2, 10, 10), dtype=float32, scope=/default)

In [124]:
r1, r2 = T.scan(lambda a,b:middletest(a,b),sequences=[ T.transpose(DATA[:, 1:]) ], init=T.stack([RNTK,GP]))

a[0] Op(name=_rewriting_take_56, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)
a[1] Op(name=_rewriting_take_57, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)
B Placeholder(name=_19, shape=(10,), dtype=float32, scope=/default)


In [125]:
def middletest(a,b):
    # print("A", a)
    print("a[0]", a[0])
    print("a[1]", a[1])
    print("B", b)
    return a,b